<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/4_KoBERT_emotion_finetune_cc_70000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [1]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000_cc.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595733 sha256=0ae0fc67c3f7257ae85c5c4e63fa58e1a20d49fedd442270603c65878f7a9d7d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 769 kB 4.2 MB/s 
     |████████████████████████████████| 3.0 MB 39.5 MB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3rtc7vz0
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3rtc7vz0
     |████████████████████████████████| 4.9 MB 4.3 MB/s 
     |████████████████████████████████| 3.4 MB 24.6 MB/s 
     |████████████████████████████████| 61 kB 530 kB/s 
     |████████████████████████████████| 3.3 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=7814e626bbbaa7a4c3e91b1afde20b00511a1a069f4eb11dae59a1ebca3265cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dumhkea/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/train
/content
popd -> /content


In [11]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/valid
/content
popd -> /content


In [12]:
train_json_list[0]

'DCNA20000213.json.answer'

In [13]:
!cat $TRAIN_DIR/{train_json_list[0]}

이번에 그 자동차 관련해서 운전면허 반납	이번에 그 자동차 관련해서 운전면허 반납	4
시스템이 생겼다고 하는데 혹시 알고 있니?	시스템이 생겼다고 하는데 혹시 알고 있니?	1
면허 반납?	면허 반납?	1
어 이번에 그 고령자들을 대상으로 면허를 반납하는	어 이번에 그 고령자들을 대상으로 면허를 반납하는	1
제도가 있었는데 그게 조금 더 확대가 되고 있는 추세인가 봐.	제도가 있었는데 그게 조금 더 확대가 되고 있는 추세인가 봐.	4
나 들었는데 그 칠십 대 이상 노인한테 한 십 만 원 정도를 준다고 하더라고	나 들었는데 그 칠십 대 이상 노인한테 한 십 만 원 정도를 준다고 하더라고	4
아 맞아 그리고 어~ 나는 이 제도가 근데 맞는 건지 모르겠어.	아 맞아 그리고 어~ 나는 이 제도가 근데 맞는 건지 모르겠어.	4
왜냐하면 그 노인 분들이 일을 하는 데에 있어서	왜냐하면 그 노인 분들이 일을 하는 데에 있어서	0
일자리를 그 운전하는 분이라면 이게 일자리를 뺏는 게 이런 게 아닐까라는 생각을 해 봤는데	일자리를 그 운전하는 분이라면 이게 일자리를 뺏는 게 이런 게 아닐까라는 생각을 해 봤는데	4
어떻게 보면 이제 노인분들의 안전을 위해서 맞는 것 같다는 생각도 들어.	어떻게 보면 이제 노인분들의 안전을 위해서 맞는 것 같다는 생각도 들어.	4
나는 이게 뭔가 강요는 아니잖아?	나는 이게 뭔가 강요는 아니잖아?	2
뭔가 유도하는 거니까 면허가 딱히 필요 없는 사람들한테는 반납을 유도하고	뭔가 유도하는 거니까 면허가 딱히 필요 없는 사람들한테는 반납을 유도하고	4
뭐 택시 기사나 이런 사람은 반납을 안 하면 되는 거니까 나는 이런 제도는 좀 괜찮다고 생각해.	뭐 택시 기사나 이런 사람은 반납을 안 하면 되는 거니까 나는 이런 제도는 쫌 괜찮다고 생각해.	4
맞아 이게 그 조건부라는 사실이 되게 중요한 거 같애.	맞아 이게 그 조건부라는 사실이 되게 중요한 거 같애.	4
거기에 조금 더 집중을 해야 되는 게 맞는 거 같고	거기에 조금 더 집중을 해야 되는 게 맞는

In [14]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,이번에 그 자동차 관련해서 운전면허 반납,이번에 그 자동차 관련해서 운전면허 반납,4
1,시스템이 생겼다고 하는데 혹시 알고 있니?,시스템이 생겼다고 하는데 혹시 알고 있니?,1
2,면허 반납?,면허 반납?,1
3,어 이번에 그 고령자들을 대상으로 면허를 반납하는,어 이번에 그 고령자들을 대상으로 면허를 반납하는,1
4,제도가 있었는데 그게 조금 더 확대가 되고 있는 추세인가 봐.,제도가 있었는데 그게 조금 더 확대가 되고 있는 추세인가 봐.,4
...,...,...,...
279,예를 들면 빽미러를 잘 봐라 이런 운전 팁 같은 거 있어?,예를 들면 빽미러를 잘 봐라 이런 운전 팁 같은 거 있어?,1
280,어~ 맞아 너 말대로 빽미러를 잘 보는 거는 정말 중요한 거 같아 실제로도 내가 운전면허,어~ 맞아 너 말대로 빽미러를 잘 보는 거는 정말 중요한 거 같아 실제로도 내가 운전면허,4
281,내가 운전면허 시험을 봤을 때 인제 빽미러를 꼭 먼저 보세요 라고 말을 해 주는 선...,내가 운전면허 시험을 봤을 때 인제 빽미러를 꼭 먼저 보세요 라고 말을 해 주는 선...,5
282,결론적으로 자동차를 운전하는 데에 있어서 안전한 제도의 도입이 너무 필요한 것 같고,결론적으로 자동차를 운전하는 데에 있어서 안전한 제도의 도입이 너무 필요한 것 같고,4


In [15]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [16]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

In [17]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

In [18]:
dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

In [19]:
print(len(dataset_train))
print(len(dataset_valid))

45000
15000


#KoBERT 입력 데이터로 변환#

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [23]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 3698, 1185, 3893, 1087, 6553, 3520, 6198, 7887, 2207, 5669,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(12, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [25]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [27]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.181636929512024 train acc 0.546875
epoch 1 batch id 201 loss 0.964531421661377 train acc 0.5891635572139303
epoch 1 batch id 401 loss 0.9543845653533936 train acc 0.5976465087281796
epoch 1 batch id 601 loss 1.0507217645645142 train acc 0.6043573211314476
epoch 1 train acc 0.6090642755681818


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6826241134751773


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.19538462162017822 train acc 0.953125
epoch 5 batch id 201 loss 0.06935851275920868 train acc 0.9468283582089553
epoch 5 batch id 401 loss 0.04492904618382454 train acc 0.9561253117206983
epoch 5 batch id 601 loss 0.1054779663681984 train acc 0.959182612312812
epoch 5 train acc 0.9599831321022727


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.6917996453900709


In [28]:
torch.save(model.state_dict(), SAVEPOINT_PATH)